Đề xuất cho người dùng các sản phẩm dựa vào sự tương quan người dùng với nhau

Part1: Xây dựng model ALS để thực hiện công việc đề xuất

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

spark-2.4.0-bin-hadoop2.7/
spark-2.4.0-bin-hadoop2.7/python/
spark-2.4.0-bin-hadoop2.7/python/setup.cfg
spark-2.4.0-bin-hadoop2.7/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/resultiterable.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/shell.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/heapq3.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/join.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/version.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/rdd.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/java_gateway.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/find_spark_home.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/_globals.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/worker.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/feature.py
spark-2.4.0-bin-hadoop2.7/python/pyspark

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/MyDrive/LDS0_K271_NGUYENMINHQUANG/Project3/'

/content/gdrive/MyDrive/LDS0_K271_NGUYENMINHQUANG/Project3


In [ ]:
import pyspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkContext

from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import isnull, when, count, col

In [ ]:
SparkContext.setSystemProperty('spark.executor.memory', '12g')
sc = SparkContext()

In [ ]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [ ]:
reviews = spark.read.csv('Reviews.csv', inferSchema=True, header=True)
products = spark.read.csv('Products.csv', inferSchema=True, header=True)

In [ ]:
reviews.printSchema()

root
 |-- id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- title: string (nullable = true)
 |-- content: string (nullable = true)



In [ ]:
reviews.show(5)

+---+-----------+----------+-----------------+------+------------------+--------------------+
| id|customer_id|product_id|             name|rating|             title|             content|
+---+-----------+----------+-----------------+------+------------------+--------------------+
|  0|     709310|  10001012| Lân Nguyễn Hoàng|     3|Ko dùng đc thẻ nhớ|Lúcđầu quên thông...|
|  1|   10701688|  10001012| Nguyễn Khánh Hòa|     5|   Cực kì hài lòng|Tiki giao hàng nh...|
|  2|   11763074|  10001012|  Toàn Phạm Khánh|     5|   Cực kì hài lòng|chất lượng camera...|
|  3|    9909549|  10001012|Nguyen Quang Minh|     5|      Rất hài lòng|Hàng được đóng gó...|
|  4|    1827148|  10001012|      Phạm Bá Đức|     5|   Cực kì hài lòng|dễ cài đặt, chất ...|
+---+-----------+----------+-----------------+------+------------------+--------------------+
only showing top 5 rows



In [ ]:
products.printSchema()

root
 |-- item_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- price: string (nullable = true)
 |-- list_price: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- group: string (nullable = true)
 |-- url: string (nullable = true)
 |-- image: string (nullable = true)



In [ ]:
products.show(5)

+--------------------+--------------------+------------------+------+-----+----------+-----+-----+----+-----+
|             item_id|                name|       description|rating|price|list_price|brand|group| url|image|
+--------------------+--------------------+------------------+------+-----+----------+-----+-----+----+-----+
|            48102821|Tai nghe Bluetoot...|THÔNG TIN CHI TIẾT|  null| null|      null| null| null|null| null|
|  Dung lượng pin 300|                null|              null|  null| null|      null| null| null|null| null|
|Thời gian pin - T...|                null|              null|  null| null|      null| null| null|null| null|
|         Bluetooth 5|                null|              null|  null| null|      null| null| null|null| null|
|     Thương hiệu OEM|                null|              null|  null| null|      null| null| null|null| null|
+--------------------+--------------------+------------------+------+-----+----------+-----+-----+----+-----+
only showi

In [ ]:
reviews = reviews.withColumn('customer_id', 
                             reviews['customer_id'].cast(IntegerType()))
reviews = reviews.withColumn('product_id', 
                             reviews['product_id'].cast(IntegerType()))
reviews = reviews.withColumn('rating', 
                             reviews['rating'].cast(IntegerType()))

In [ ]:
reviews.count()

362778

In [ ]:
nacounts = reviews.select([count(when(isnull(c), c)).alias(c) for c in reviews.columns]).toPandas()
nacounts

,id,customer_id,product_id,name,rating,title,content
0,32,1658,1688,1902,1718,1693,197730


In [ ]:
reviews = reviews.na.drop(subset=['customer_id', 'product_id', 'rating'], how='any')

In [ ]:
reviews.count()

361060

In [ ]:
nacounts = reviews.select([count(when(isnull(c), c)).alias(c) for c in reviews.columns]).toPandas()
nacounts

,id,customer_id,product_id,name,rating,title,content
0,0,0,0,428,0,28,196039


In [ ]:
# Distinct users and products
users = reviews.select('customer_id').distinct().count()
prods = reviews.select('product_id').distinct().count()
numerator = reviews.count()

In [ ]:
display(numerator, users, prods)

361060

251125

4214

In [ ]:
#Number od ratings matrix could contain if no empty cells
denominator = users * prods
denominator

1058240750

In [ ]:
# Calculating sparsity
sparsity = 1 - (numerator*1.0 / denominator)
print('Sparsity: '), sparsity

Sparsity: 


(None, 0.9996588110975693)

In [ ]:
# Smaller dataset so we will use 0.8 / 0.2
reviews_sub = reviews.select('customer_id', 'product_id', 'rating')
reviews_sub.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [ ]:
reviews_sub.summary().show()

+-------+-----------------+--------------------+------------------+
|summary|      customer_id|          product_id|            rating|
+-------+-----------------+--------------------+------------------+
|  count|           361060|              361060|            361060|
|   mean|9179036.923461474| 2.440459540852213E7| 4.475092782363042|
| stddev|6307841.885469688|2.3784665258464653E7|1.0167026465603923|
|    min|               10|               54665|                 1|
|    25%|          2121652|             1600005|                 4|
|    50%|          8579793|            15088370|                 5|
|    75%|         14493364|            47321729|                 5|
|    max|         21013443|            81964004|                 5|
+-------+-----------------+--------------------+------------------+



In [ ]:
(training, test) = reviews_sub.randomSplit([0.8, 0.2])

In [ ]:
training.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [ ]:
als = ALS(maxIter=15,
          regParam=0.1,
          userCol='customer_id',
          itemCol='product_id',
          ratingCol='rating',
          coldStartStrategy='drop',
          nonnegative=True)
model = als.fit(training)

In [ ]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [ ]:
evaluator = RegressionEvaluator(metricName='rmse',
                                labelCol='rating',
                                predictionCol='prediction')

In [ ]:
rmse = evaluator.evaluate(predictions)
print('Root-mean-square error = ' + str(rmse))

Root-mean-square error = 1.3740798869877568


# Providing Recommendation: for all users

In [ ]:
# Get 10 recommendations which have highest rating
user_recs = model.recommendForAllUsers(5)

In [ ]:
user_recs.printSchema()

root
 |-- customer_id: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- product_id: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [ ]:
for user in user_recs.head(3):
  print(user)

Row(customer_id=833, recommendations=[Row(product_id=70720181, rating=6.317577838897705), Row(product_id=2080951, rating=6.001999855041504), Row(product_id=74556923, rating=5.846463203430176), Row(product_id=8141046, rating=5.828258514404297), Row(product_id=69149307, rating=5.807104587554932)])
Row(customer_id=7993, recommendations=[Row(product_id=16966882, rating=6.325831413269043), Row(product_id=73682812, rating=6.2319769859313965), Row(product_id=21095699, rating=6.17805814743042), Row(product_id=52069061, rating=6.098493576049805), Row(product_id=10736184, rating=6.090983867645264)])
Row(customer_id=17420, recommendations=[Row(product_id=52699359, rating=6.790926456451416), Row(product_id=71087291, rating=6.3270263671875), Row(product_id=28266428, rating=6.229225158691406), Row(product_id=76466253, rating=6.059597969055176), Row(product_id=53833095, rating=6.039615154266357)])


# Give recommendation for a specific user

In [ ]:
# Products that userId bought
userId = 6177374
test.filter(test['customer_id']==userId).sort('rating', asceding=False).show()

+-----------+----------+------+
|customer_id|product_id|rating|
+-----------+----------+------+
|    6177374|    416613|     5|
|    6177374|  32033717|     5|
|    6177374|  23556574|     5|
|    6177374|  47452735|     5|
|    6177374|  52070229|     5|
|    6177374|  54017350|     5|
+-----------+----------+------+



In [ ]:
training.filter(test['customer_id']==userId).sort('rating', 
                                                  asceding=False).show()

+-----------+----------+------+
|customer_id|product_id|rating|
+-----------+----------+------+
|    6177374|    702132|     5|
|    6177374|    845378|     5|
|    6177374|   7817447|     5|
|    6177374|  71051598|     5|
|    6177374|  71197117|     5|
|    6177374|  71198812|     5|
|    6177374|  75186039|     5|
|    6177374|   4497817|     5|
|    6177374|  28571379|     5|
|    6177374|  32033717|     5|
|    6177374|  35726089|     5|
|    6177374|  38458616|     5|
|    6177374|  38458616|     5|
|    6177374|  44009404|     5|
|    6177374|  15239985|     5|
|    6177374|  23459272|     5|
|    6177374|    555019|     5|
|    6177374|  47321729|     5|
|    6177374|  47499193|     5|
|    6177374|  49661643|     5|
+-----------+----------+------+
only showing top 20 rows



In [ ]:
# Example: Recommender for customer_id = 6177374
userId = 6177374
result = user_recs.filter(user_recs['customer_id']==userId)
result.show(truncate=False)

+-----------+----------------------------------------------------------------------------------------------------------------+
|customer_id|recommendations                                                                                                 |
+-----------+----------------------------------------------------------------------------------------------------------------+
|6177374    |[[70720181, 6.9441333], [72119221, 6.7197785], [71087291, 6.7024393], [66153264, 6.6046], [66251373, 6.5120916]]|
+-----------+----------------------------------------------------------------------------------------------------------------+



In [ ]:
from pyspark.sql.functions import col, explode

In [ ]:
result = result.select(result.customer_id, 
                       explode(result.recommendations))

In [ ]:
result.show()

+-----------+--------------------+
|customer_id|                 col|
+-----------+--------------------+
|    6177374|[70720181, 6.9441...|
|    6177374|[72119221, 6.7197...|
|    6177374|[71087291, 6.7024...|
|    6177374|  [66153264, 6.6046]|
|    6177374|[66251373, 6.5120...|
+-----------+--------------------+



In [ ]:
result = result.withColumn('product_id', 
                           result.col.getField('product_id'))\
                .withColumn('rating', 
                            result.col.getField('rating'))
result.show()

+-----------+--------------------+----------+---------+
|customer_id|                 col|product_id|   rating|
+-----------+--------------------+----------+---------+
|    6177374|[70720181, 6.9441...|  70720181|6.9441333|
|    6177374|[72119221, 6.7197...|  72119221|6.7197785|
|    6177374|[71087291, 6.7024...|  71087291|6.7024393|
|    6177374|  [66153264, 6.6046]|  66153264|   6.6046|
|    6177374|[66251373, 6.5120...|  66251373|6.5120916|
+-----------+--------------------+----------+---------+



In [ ]:
# Filter all products having rating > 4.0
result.filter(result.rating>=4.0).show()

+-----------+--------------------+----------+---------+
|customer_id|                 col|product_id|   rating|
+-----------+--------------------+----------+---------+
|    6177374|[70720181, 6.9441...|  70720181|6.9441333|
|    6177374|[72119221, 6.7197...|  72119221|6.7197785|
|    6177374|[71087291, 6.7024...|  71087291|6.7024393|
|    6177374|  [66153264, 6.6046]|  66153264|   6.6046|
|    6177374|[66251373, 6.5120...|  66251373|6.5120916|
+-----------+--------------------+----------+---------+



In [ ]:
# Save to disk
user_recs.write.parquet('user_recs.parquet', mode='overwrite')